In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# 1. WCZYTANIE DANYCH

try:
    df = pd.read_csv("AMES_housing_Price.csv")
except FileNotFoundError:

    df = pd.read_csv("data/AMES_housing_Price.csv")

print(f"Wczytano {len(df)} wierszy danych.")


# 2. SELEKCJA ZMIENNYCH (FEATURE ENGINEERING)
# GrLivArea   -> Odpowiednik powierzchni mieszkalnej (Above grade living area)
# OverallQual -> Odpowiednik jakości wykończenia (1-10)
# YearBuilt   -> Rok budowy (wiek domu wpływa na cenę)
# BldgType    -> Typ budynku (np. jednorodzinny, bliźniak)

predictors_num = ['GrLivArea', 'OverallQual', 'YearBuilt']
predictors_cat = ['BldgType']
target = 'SalePrice'

all_cols = predictors_num + predictors_cat + [target]

# 3. CZYSZCZENIE DANYCH

data = df[all_cols].dropna().reset_index(drop=True)

print(f"Dane po czyszczeniu: {len(data)} wierszy")

# 4. PRZETWARZANIE ZMIENNYCH (ENCODING)

data_dummies = pd.get_dummies(data[predictors_cat], drop_first=True)

X = pd.concat([data[predictors_num], data_dummies], axis=1)
y = data[target]

print("\nZmienne użyte w modelu (X):")
print(list(X.columns))

# 5. PODZIAŁ NA ZBIÓR TRENINGOWY I TESTOWY

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 6. TRENING MODELU

model = LinearRegression()
model.fit(X_train, y_train)

# 7. EWALUACJA (OCENA MODELU)

y_pred = model.predict(X_test)

# Metryki
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print("-" * 50)
print("WYNIKI MODELU (na zbiorze testowym):")
print("-" * 50)
print(f"RMSE (Średni błąd predykcji): ${rmse:,.0f}")
print(f"R^2 (Współczynnik dopasowania): {r2:.4f}")
print("-" * 50)

# 8. INTERPRETACJA WSPÓŁCZYNNIKÓW

print("\nCO MÓWI MODEL? (Interpretacja biznesowa):")
print(f"Stała (Intercept): ${model.intercept_:,.0f} (Teoretyczna cena bazowa)")

coefficients = pd.DataFrame({
    'Zmienna': X.columns,
    'Wpływ ($)': model.coef_
}).sort_values(by='Wpływ ($)', ascending=False)

print(coefficients)
print("\nLegenda:")
print("- GrLivArea: O ile wzrośnie cena za każdą dodatkową stopę kwadratową powierzchni.")
print("- OverallQual: O ile wzrośnie cena za każdy punkt jakości (w skali 1-10).")
print("- YearBuilt: O ile wzrośnie cena za każdy rok 'młodszy' dom.")
print("- Zmienne typu BldgType_...: Różnica w cenie względem typu bazowego (zazwyczaj 1Fam).")

Wczytano 1460 wierszy danych.
Dane po czyszczeniu: 1460 wierszy

Zmienne użyte w modelu (X):
['GrLivArea', 'OverallQual', 'YearBuilt', 'BldgType_2fmCon', 'BldgType_Duplex', 'BldgType_Twnhs', 'BldgType_TwnhsE']
--------------------------------------------------
WYNIKI MODELU (na zbiorze testowym):
--------------------------------------------------
RMSE (Średni błąd predykcji): $42,346
R^2 (Współczynnik dopasowania): 0.7662
--------------------------------------------------

CO MÓWI MODEL? (Interpretacja biznesowa):
Stała (Intercept): $-1,198,123 (Teoretyczna cena bazowa)
           Zmienna     Wpływ ($)
1      OverallQual  24229.818202
2        YearBuilt    581.773582
0        GrLivArea     57.845110
3  BldgType_2fmCon  -6651.425266
6  BldgType_TwnhsE -20981.043872
4  BldgType_Duplex -22371.848954
5   BldgType_Twnhs -45163.243440

Legenda:
- GrLivArea: O ile wzrośnie cena za każdą dodatkową stopę kwadratową powierzchni.
- OverallQual: O ile wzrośnie cena za każdy punkt jakości (w skali 